# Fundamentos de Projeto de Controladores

----
Este material serve apenas como um guia para estudos e para as aulas, não substituindo a leitura dos livros indicados e é de uso exclusivo dos alunos do CEFET/RJ campus Angra dos Reis.

---

## Sintonia de Controladores

* aumentar o ganho proporcional $K_P$ tende a tornar a resposta mais rápida, mas pode aumentar o sobrevalor;
* aumentar o ganho integral $K_I$ tende a reduzir o erro em regime permanente, mas pode tornar a resposta mais lenta;
* aumentar o ganho derivativo $K_D$ tende a reduzir o sobrevalor e o tempo de assentamento. 

**implementar um controlador PID requer selecionar três ganhos independentes de modo a atingir um bom equilíbrio dos critérios de desempenho da malha fechada tais como tempo de resposta, sobrevalor, tempo de assentamento e erro em regime permanente.** 



Ziegler e Nichols desenvolveram dois métodos para seleção de "bons" ganhos PID. 

Essas "regras de sintonia dos PID" foram baseadas em tentativas heurísticas conduzidas por Ziegler e Nichols e elas fornecem aos engenheiros de controle um bom ponto de partida para a seleção dos ganhos de um PID que possibilitam um desempenho satisfatório da malha fechada. 

Os parâmetros-chave da curva de reação são tempo de retardo $T_d$ e a inclinação $R$. 

A curva de reação é a resposta do sistema de malha aberta a uma entrada do tipo degrau. 

Ziegler e Nichols usaram esses dois parâmetros para determinar os ganhos PID que produzem uma resposta em malha fechada que exibe uma razão de decaimento de um quarto, ou seja, a resposta transiente decai a um quarto do seu valor de pico em um período de oscilação. 

![](proj_cont_1.png)


Regras de sintonia dos ganhos para os controladores P, PI e PID. 

![](proj_cont_2.png)


O segundo método de sintonia dos PID desenvolvido por Ziegles e Nichols depende da obtenção de uma resposta em malha fechada **marginalmente estável** com um ajuste de elevados ganhos. 

Nessa técnica, o ganho do controlador P é continuamente aumentado até que a resposta da malha fechada passe de oscilações senoidais amortecidas para oscilações permanentes com amplitude constante. 

Ziegler e Nichols denominaram o ajuste do ganho P que resulta nas oscilações permanentes de "ganho final" $K_F$. 

O período das oscilações permanentes é $P_F$ (o "período final").


![](proj_cont_3.png)

Em resumo, as regras de ajuste de Ziegler-Nichols permitem o engenheiro de controle obter o bom ponto de partida para selecionar os ganhos do PID. O projeto final do controlador PID pode ser realizado por meio de testes experimentais complementares ou simulações com ajustes adicionais de um ganho específico de modo a melhorar determinado aspecto da resposta transiente.

Deve-se notar que o método de sintonia de Ziegler-Nichols não se aplica necessariamente a todos os sistemas físicos. Nem todas as plantas exibem uma resposta à entrada degrau na forma de S ou podem ser comandadas de modo a manter oscilações permanentes pelo aumento do ganho proporcional.

## Estabilidade de Malha Fechada

Estabilidade é um atributo essencial para um sistema de controle de malha fechada. 

Espera-se que um sistema de controle em malha fechada estável permaneça "sob controle" durante todas as condições normais de operação.  

Será empregada a definição de estabilidade denominada **entrada limitada, saída limitada** (BIBO - "*Bounded-Input, Bounded-Output*"): um sistema é BIBO estável se para toda entrada limitada a saída permanece limitada para todos os instantes de tempo. 

**Para sistemas lineares, invariantes no tempo (LIT), a estabilidade BIBO requer que todas as raízes características (ou polos ou autovalores) estejam no lado esquerdo do plano complexo.**  

**Exemplo 1:** 

Análise a estabilidade do seguinte sistema: 

$T(s) = \dfrac{K_P G_P (s)}{1 + K_P G_P(s)} =\dfrac{K_P}{0,5s^3 + 4s^2 + 23s + K_P}$

In [1]:
from control import *

In [2]:
Kp = 30

sistema = tf([1],[0.5,4,23,34 + Kp])
sistema

TransferFunction(array([1]), array([ 0.5,  4. , 23. , 64. ]))

In [3]:
poles(sistema)

array([-1.87136313+5.15404596j, -1.87136313-5.15404596j,
       -4.25727373+0.j        ])

Observa-se que os 3 polos do sistema possuem raíz com a parte real negativa (estando no lado esquerdo do plano complexo), logo o sistema é BIBO estável. 

In [4]:
lista_Kp = [1,50,100,150,200,250]

In [5]:
print('Kp -> Polos')
for i in lista_Kp:
    sistema = tf([1],[0.5,4,23,34 + i])
    polos = poles(sistema)
    print(str(i) + ' -> ' + str(polos))

Kp -> Polos
1 -> [-2.96131716+4.99270702j -2.96131716-4.99270702j -2.07736569+0.j        ]
50 -> [-5.30094552+0.j        -1.34952724+5.4654584j -1.34952724-5.4654584j]
100 -> [-6.93765069+0.j         -0.53117466+6.19254754j -0.53117466-6.19254754j]
150 -> [-8.00000000e+00+0.j         -3.55271368e-15+6.78232998j
 -3.55271368e-15-6.78232998j]
200 -> [-8.80906405+0.j          0.40453202+7.27759925j  0.40453202-7.27759925j]
250 -> [-9.4733564+0.j          0.7366782+7.70810842j  0.7366782-7.70810842j]


| Ganho do Controlador, $K_p$ | Polos de Malha Fechada | Condição de Estabilidade |
| --- | --- | --- |
| 50 | $ s_1 = -5.3j$, $s_{2,3} = -2.96 \pm 4.99j $| Estável |
| 100 | $ s_1 = -5.3009$ , $ s_{2,3} = -1.34952 \pm 5.465j $| Estável |
| 150 | $ s_1 = -8.0 $,$ s_{2,3} = \pm 6.7823j $ | Marginalmente estável |
| 200 | $ s_1 = -8.8091 $,$ s_{2,3} = 0.4045 \pm 7.2776j $ | Instável |
| 250 | $ s_1 = -9.4734 $, $s_{2,3} = 0.7367 \pm 7.7081j $| Instável |

* Critério de estabilidade de Routh-Hurwitz

### Método do Lugar das Raízes

In [6]:
sistema = tf([1],[0.5,4,23,34])

In [7]:
import matplotlib.pyplot as plt

In [8]:
%matplotlib auto
plt.figure(dpi=150)
raizes,ganho = root_locus(sistema,print_gain = True,plot = True)

Using matplotlib backend: <object object at 0x000001DD77C92120>


In [9]:
import pandas as pd

In [10]:
pd.DataFrame({'Raíz 1':raizes[:,0],'Raíz 2':raizes[:,1],'Raíz 3':raizes[:,2],'Ganho':ganho})

,Raíz 1,Raíz 2,Raíz 3,Ganho
0,-3.0000000-5.0000000j,-3.0000000+5.0000000j,-2.0000000+0.0000000j,0.000000e+00
1,-2.6666670-4.9665550j,-2.6666670+4.9665550j,-2.6666670+0.0000000j,8.370370e+00
2,-2.6666670-4.9665550j,-2.6666670+4.9665550j,-2.6666670+0.0000000j,8.370370e+00
3,-1.9304630-5.1276360j,-1.9304630+5.1276360j,-4.1390750+0.0000000j,2.812615e+01
4,-1.3968630-5.4317470j,-1.3968630+5.4317470j,-5.2062740+0.0000000j,4.788193e+01
...,...,...,...,...
56,7.283980-017.9363530j,7.283980+017.9363530j,-22.567960+0.000000j,4.194887e+03
57,7.355531-018.0554700j,7.355531+018.0554700j,-22.711062+0.000000j,4.282281e+03
58,19.278294-38.332891j,19.278294+38.332891j,-46.556587+0.000000j,4.282281e+04
59,44.772759-82.317459j,44.772759+82.317459j,-97.545518+0.000000j,4.282281e+05


In [11]:
sistema2 = tf([1,6],[2,4,0])
sistema2

TransferFunction(array([1, 6]), array([2, 4, 0]))

In [12]:
plt.figure(dpi=150)
root_locus(sistema2,xlim=(-16,0));